In [ ]:
import warnings
warnings.filterwarnings("ignore")   


In [ ]:
print("RAG Chatbot Project Started Successfully")    


RAG Chatbot Project Started Successfully


In [ ]:
%pip install langchain==0.1.16 langchain-community==0.0.36 langchain-core==0.1.48



In [ ]:
#%pip install langchain-community langchain-core langchain

  Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
  Using cached langchain_core-1.2.4-py3-none-any.whl (477 kB)
  Using cached langchain-1.2.0-py3-none-any.whl (102 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import langchain_community
print("LangChain Community installed successfully")   


LangChain Community installed successfully


In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", temperature=0.2)

response = llm.invoke("Reply with just one word: READY")
print(response)


 Affirmative.


In [ ]:
llm.invoke("Reply with exactly this word and nothing else: READY")    

' READY'

In [ ]:
%pip install pypdf       
%pip install python-docx
%pip install python-pptx
%pip install unstructured
%pip install sentence-transformers
%pip install faiss-cpu





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    Docx2txtLoader,
    UnstructuredPowerPointLoader
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [ ]:
from langchain.schema import Document


In [ ]:
def load_excel_as_text(file_path):
    all_text = []

    excel_file = pd.ExcelFile(file_path)

    for sheet_name in excel_file.sheet_names:
        df = excel_file.parse(sheet_name)

        sheet_text = f"Sheet name: {sheet_name}\n"
        sheet_text += df.astype(str).to_string(index=False)

        all_text.append(sheet_text)

    return "\n\n".join(all_text)


In [ ]:
import pandas as pd

def load_csv_as_text(file_path):
    df = pd.read_csv(file_path)

    text = df.to_string(index=False)

    return text


In [ ]:
def load_documents(folder_path):
    documents = []

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)

        if file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())

        elif file.endswith(".txt"):
            loader = TextLoader(file_path)
            documents.extend(loader.load())

        elif file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
            documents.extend(loader.load())

        elif file.endswith(".pptx"):
            loader = UnstructuredPowerPointLoader(file_path)
            documents.extend(loader.load())

        elif file.endswith((".xls", ".xlsx")):
            excel_text = load_excel_as_text(file_path)
            doc = Document(
                page_content=excel_text,
                metadata={"source": file_path}
            )
            documents.append(doc)

        elif file.endswith(".csv"):
            csv_text = load_csv_as_text(file_path)
            doc = Document(
                page_content=csv_text,
                metadata={"source": file_path}
            )
            documents.append(doc)

        else:
            continue

    return documents


In [ ]:
import os

In [ ]:
%pip install docx2txt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
folder_path = "data"

documents = load_documents(folder_path)
print("Total documents loaded:", len(documents))


Total documents loaded: 334


In [ ]:
print(documents[0].page_content[:300])
print(documents[0].metadata)




{'source': 'data\\big data.pdf', 'page': 0}


In [ ]:
%pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
print(os.listdir("data"))


['big data.pdf', 'Constructor Info.txt', 'lightgbm.pptx', 'viva.docx', 'ZAir Quality.csv']


In [ ]:
for doc in documents[-10:]:
    print(doc.metadata["source"])


data\big data.pdf
data\big data.pdf
data\big data.pdf
data\big data.pdf
data\big data.pdf
data\big data.pdf
data\Constructor Info.txt
data\lightgbm.pptx
data\viva.docx
data\ZAir Quality.csv


In [ ]:
csv_docs = [
    doc for doc in documents
    if doc.metadata["source"].lower().endswith(".csv")
]

print("CSV documents found:", len(csv_docs))

if csv_docs:
    print(csv_docs[0].page_content[:1500])


CSV documents found: 1
 Unique ID  Indicator ID                                                   Name                              Measure         Measure Info Geo Type Name  Geo Join ID                                 Geo Place Name         Time Period Start_Date  Data Value  Message
    336867           375                                 Nitrogen dioxide (NO2)                                 Mean                  ppb            CD          407                  Flushing and Whitestone (CD7)      Winter 2014-15 12/01/2014   23.970000      NaN
    336741           375                                 Nitrogen dioxide (NO2)                                 Mean                  ppb            CD          107                          Upper West Side (CD7)      Winter 2014-15 12/01/2014   27.420000      NaN
    550157           375                                 Nitrogen dioxide (NO2)                                 Mean                  ppb            CD          414              Rockawa

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=400,
    chunk_overlap=50

)


In [ ]:
chunks = text_splitter.split_documents(documents)


In [ ]:
print("Total chunks created:", len(chunks))


Total chunks created: 21025


In [ ]:
chunks[0]


Document(page_content='Big Data Analytics', metadata={'source': 'data\\big data.pdf', 'page': 1})

In [ ]:

print(chunks[0].page_content)


Big Data Analytics


In [ ]:
chunks[0].metadata


{'source': 'data\\big data.pdf', 'page': 1}

step 2.5

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.vectorstores import FAISS


In [ ]:
vector_db = FAISS.from_documents(chunks, embeddings)

vector_db.save_local("vectorstore")


In [ ]:
print("Vector DB created with", vector_db.index.ntotal, "vectors")


Vector DB created with 21025 vectors


In [ ]:
query = "What is big data?"

results = vector_db.similarity_search(query, k=3)


In [ ]:
for i, doc in enumerate(results):
    print(f"\nResult {i+1}")
    print(doc.page_content[:300])
    print("Source:", doc.metadata)



Result 1
more agile to face the demand and competitions. For example, a lot of decisions have to be made in 
real time during the sale on Thanksgiving day. Big data takes the form of messages and images posted 
to social networks; readings from sensors; GPS signals from cell phones; and more. There is huge v
Source: {'source': 'data\\big data.pdf', 'page': 7}

Result 2
(Data) that grows everyday (Big Data). We 
need to analyze this to get information.
• The problems in handling Big Data by 
traditional database systems are addressed 
here and the need for different technologies 
to analyze big data is understood.
• An overview of the technologies that support 
in 
Source: {'source': 'data\\big data.pdf', 'page': 30}

Result 3
Preface • ix
given at the end of the chapters can be used to test the readers understanding of the content provided 
in the chapter. Further, a list of suggested programming assignments can be used by a mature reader to 
gain expertise in this field. 
  Organizat

In [ ]:
from langchain_community.llms import Ollama


In [ ]:
llm = Ollama(
    model="mistral",
    temperature=0.2,
    num_ctx=2048
)



In [ ]:
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)


In [ ]:
import re

def format_answer_sentence_wise(text):
    
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    sentences = [s.strip() for s in sentences if s.strip()]

    return "\n".join(sentences)


In [ ]:
from langchain.prompts import PromptTemplate


In [ ]:
qa_cache = {}


In [ ]:
def ask_question(question):
   
    if question in qa_cache:
        return qa_cache[question]

    
    docs = retriever.get_relevant_documents(question)
    docs_and_scores = vector_db.similarity_search_with_score(question, k=3)

       
    RELEVANCE_THRESHOLD = 0.6   
    filtered_docs = [
        doc for doc, score in docs_and_scores if score < RELEVANCE_THRESHOLD
    ]

    
    if not filtered_docs:
        answer = "The given context does not contain information related to this question."
        return answer, set()

    
    context = "\n\n".join(doc.page_content for doc in docs)

    
    prompt = f"""
Answer the question strictly using the given context.
Do not add extra information.

Context:
{context}

Question:
{question}

Answer:
"""

    
    answer = llm.invoke(prompt)

    
    sources = {
        f"{doc.metadata.get('source')} (page {doc.metadata.get('page', 'N/A')})"
        for doc in docs
    }

    
    qa_cache[question] = (answer, sources)

    return answer, sources




In [43]:
question = "What is Big Data?"

answer, sources = ask_question(question)

print("Answer:\n", answer)
print("\nSources:")
for s in sources:
    print("-", s)


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer:
  Big Data refers to a large volume of data that grows everyday, taking the form of messages and images posted to social networks; readings from sensors; GPS signals from cell phones; and more.

Sources:
- data\big data.pdf (page 30)
- data\big data.pdf (page 7)


In [46]:
import time

start = time.time()
docs = retriever.get_relevant_documents("types of constructors")
print("Retriever time:", time.time() - start)


Retriever time: 0.05852246284484863


In [47]:
import os
os.listdir("vectorstore")


['index.faiss', 'index.pkl']

In [48]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.
